In [ ]:
import pandas as pd 
import seaborn as sns
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.datasets import make_classification
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd 
data = pd.read_csv("data.csv")


In [ ]:
df = data.copy()
df = df.drop(['id', 'Unnamed: 32'], axis=1)
df.head()

In [ ]:
df.rename(columns={'concave points_worst': 'concave_points_worst',
                   'concave points_mean': 'concave_points_mean',
                   'concave points_se': 'concave_points_se'}, inplace=True)

In [ ]:
data.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.diagnosis.unique()

In [ ]:
df['diagnosis'] = df['diagnosis'].replace({'B': 0, 'M': 1})

In [ ]:
df["diagnosis"].value_counts()

In [ ]:
tanı = df['diagnosis'].value_counts().reset_index()
tanı

In [ ]:
plt.figure(figsize=(8, 6))
ax = sns.barplot(x='diagnosis', y='count', data=tanı, palette='viridis')

for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', fontsize=11, color='black', xytext=(0, 5),
                textcoords='offset points')

plt.xlabel('Tanı')
plt.ylabel('Frekans')
plt.title('Tanı Sayısı')
plt.show()

In [ ]:
plt.figure(figsize=(20,15))
plotnumber=1
for column in df:
    if plotnumber<=31:
        ax = plt.subplot(6,6, plotnumber)
        sns.distplot(df[column])
        plt.xlabel(column)
    plotnumber+=1

plt.tight_layout()
plt.show()

In [ ]:
plt.hist(df['diagnosis'], bins=10, color='skyblue', edgecolor='black')

# Histogram başlığı ve eksen etiketleri
plt.title('Öznitelik Dağılımı')
plt.xlabel('Değerler')
plt.ylabel('Frekans')

# Histogramı göster
plt.show()

In [ ]:
sns.kdeplot(df['diagnosis'], shade=True)

# Yoğunluk grafiği başlığı ve eksen etiketleri
plt.title('Öznitelik Yoğunluk Dağılımı')
plt.xlabel('Değerler')
plt.ylabel('Yoğunluk')

# Yoğunluk grafiğini göster
plt.show()

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(20, 12))
sns.heatmap(df.corr(), annot=True, cmap='viridis', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Kategorik değişkenin sıklığını hesapla
kategori_sayisi = df['diagnosis'].value_counts()

# Pasta grafiği oluştur
kategori_sayisi.plot(kind='pie', autopct='%1.1f%%', colors=['skyblue', 'orange', 'green'])

# Grafik başlığı
plt.title('Kategorik Değişkenin Dağılımı')

# Pasta grafiğini göster
plt.show()

In [ ]:
corr_matrix = df.corr().abs()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
tri_df = corr_matrix.mask(mask)

to_drop = [x for x in tri_df.columns if any(tri_df[x]>0.92)]

df = df.drop(to_drop, axis=1)

print(df.shape[1])

In [ ]:
y = df["diagnosis"]
X = df.drop(["diagnosis"], axis = 1)
columns = X.columns.tolist()

In [ ]:
lof = LocalOutlierFactor()
lof.fit(X)

# Get the negative outlier factor scores
X_score = lof.negative_outlier_factor_

# Create a scatter plot of the data points
plt.figure(figsize=(8, 6))
plt.scatter(X.iloc[:, 0], X.iloc[:, 1], color="k", s=3, label="Data Point")

# Calculate the radius for each point based on the outlier score
radius = (X_score.max() - X_score) / (X_score.max() - X_score.min())

# Plot the points with sizes corresponding to the outlier score
plt.scatter(X.iloc[:, 0], X.iloc[:, 1], s=1000 * radius, edgecolors="r", facecolors="none", label="Outlier Score")

plt.xlabel(columns[0])
plt.ylabel(columns[1])
plt.legend()
plt.grid(True)
plt.title('Outlier Detection using Local Outlier Factor (LOF)')
plt.show()

# Threshold for outlier detection
threshold_outlier = -2.5
filtre = X_score < threshold_outlier
outlier_index = np.where(filtre)[0]
print("Outlier indices:", outlier_index)

In [ ]:
plt.figure(figsize=(10, 10))

# Plot outliers in blue
plt.scatter(X.iloc[outlier_index, 0], X.iloc[outlier_index, 1], color="blue", s=50, label="Outliers")

# Plot all data points
plt.scatter(X.iloc[:, 0], X.iloc[:, 1], color="k", s=3, label="Data Point")

# Plot outlier scores with sizes corresponding to the outlier score
plt.scatter(X.iloc[:, 0], X.iloc[:, 1], s=1000 * radius, edgecolors="r", facecolors="none", label="Outlier Score")

plt.xlabel(columns[0])
plt.ylabel(columns[1])
plt.legend()
plt.grid(True)
plt.title('Outlier Detection using Local Outlier Factor (LOF)')
plt.show()

In [ ]:
X = X.drop(outlier_index)
y = y.drop(outlier_index).values

In [ ]:
X_standardized = (X - X.mean()) / X.std()

# Apply PCA without specifying the number of components
pca = PCA()
pca.fit(X_standardized)

# Calculate explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_

# Find the optimal number of components
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)
optimal_components = np.argmax(cumulative_variance_ratio >= 0.95) + 1

# Plot explained variance ratio
plt.figure(figsize=(8, 6))
plt.plot(np.cumsum(explained_variance_ratio), marker='o', linestyle='-')
plt.xlabel('Bileşen Sayısı')
plt.ylabel('Kümülatif Açıklanan Varyans Oranı')
plt.title('PCA Açıklanan Varyans Oranı')

# Annotate the plot with the optimal number of components
plt.annotate(f'Optimal Components: {optimal_components}', 
             xy=(optimal_components, cumulative_variance_ratio[optimal_components-1]),
             xytext=(optimal_components + 2, cumulative_variance_ratio[optimal_components-1] - 0.05),
             arrowprops=dict(facecolor='black', arrowstyle='->'),
             fontsize=12)

plt.grid(True)
plt.show()

In [ ]:
pca = PCA(n_components=10)
principal_components = pca.fit_transform(X_standardized)

# Create a DataFrame to hold the principal components
principal_df = pd.DataFrame(data=principal_components, columns=[f'PC{i}' for i in range(1, 11)])

# Concatenate principal components with the target variable
final_df = pd.concat([principal_df, pd.DataFrame(y, columns=['target'])], axis=1)

# Plot the PCA result
plt.figure(figsize=(8, 6))
sns.scatterplot(x='PC1', y='PC2', hue='target', data=final_df, palette='Set1')
plt.title('PCA Görselleştirme (Optimal Bileşenler)')
plt.xlabel('Ana Bileşen 1')
plt.ylabel('Ana Bileşen 2')
plt.legend(title='Target', loc='upper right')
plt.show()

In [ ]:
df.info()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train.shape

In [ ]:
loj_model = LogisticRegression()
loj_model.fit(X_train, y_train)

In [ ]:
y_pred = loj_model.predict(X_test)

In [ ]:
y_pred

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
y_test

In [ ]:
y_pred


In [ ]:
from sklearn.metrics import confusion_matrix

# Modelinizin tahminlerini kullanarak confusion matrix'i hesaplayın
conf_matrix = confusion_matrix(y_test, y_pred)

# Oluşturulan confusion matrix'i ekrana yazdırın
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix (Logistic Regression)')
plt.colorbar()
sınıf = ['İyi Huylu (0)', 'Kötü Huylu (1)']
# Sütun ve satır etiketleri
plt.xticks(np.arange(len(sınıf)), sınıf)
plt.yticks(np.arange(len(sınıf)), sınıf)

# Değerlerin gösterilmesi
for i in range(len(sınıf)):
    for j in range(len(sınıf)):

        plt.text(j, i, str(conf_matrix[i, j]), horizontalalignment='center', color='white' if conf_matrix[i, j] > conf_matrix.max() / 2 else 'black')
plt.xlabel('Tahmin Edilen Değer')
plt.ylabel('Gerçek Değer')
plt.show()

In [ ]:
y_pred_test = loj_model.predict_proba(X_test)[:, 1]
fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_pred_test)
roc_auc_test = roc_auc_score(y_test, y_pred_test)
plt.figure(figsize=(8, 6))
plt.plot(fpr_test, tpr_test, label=f'AUC (Test) = {roc_auc_test:.2f}')
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve (Test)')
plt.legend(loc='lower right')
plt.show()

In [ ]:
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

In [ ]:
y_pred = nb_model.predict(X_test)

In [ ]:
y_pred

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix (Naive Bayes)')
plt.colorbar()
sınıf = ['İyi Huylu (0)', 'Kötü Huylu (1)']
# Sütun ve satır etiketleri
plt.xticks(np.arange(len(sınıf)), sınıf)
plt.yticks(np.arange(len(sınıf)), sınıf)

# Değerlerin gösterilmesi
for i in range(len(sınıf)):
    for j in range(len(sınıf)):
        plt.text(j, i, str(conf_matrix[i, j]), horizontalalignment='center', color='white' if conf_matrix[i, j] > conf_matrix.max() / 2 else 'black')

plt.xlabel('Tahmin Edilen Değer')
plt.ylabel('Gerçek Değer')
plt.show()

In [ ]:
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)

In [ ]:
y_pred = knn_model.predict(X_test)

In [ ]:
y_pred

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix (KNN)')
plt.colorbar()
sınıf = ['İyi Huylu (0)', 'Kötü Huylu (1)']
# Sütun ve satır etiketleri
plt.xticks(np.arange(len(sınıf)), sınıf)
plt.yticks(np.arange(len(sınıf)), sınıf)

# Değerlerin gösterilmesi
for i in range(len(sınıf)):
    for j in range(len(sınıf)):
        plt.text(j, i, str(conf_matrix[i, j]), horizontalalignment='center', color='white' if conf_matrix[i, j] > conf_matrix.max() / 2 else 'black')

plt.xlabel('Tahmin Edilen Değer')
plt.ylabel('Gerçek Değer')
plt.show()

In [ ]:
knn = KNeighborsClassifier()
knn_params = {"n_neighbors": np.arange(1, 10)}

In [ ]:
knn_cv = GridSearchCV(knn, knn_params, cv=10)

In [ ]:
knn_cv.fit(X_train, y_train)

In [ ]:
print("En iyi parametreler:", knn_cv.best_params_)
print("En iyi skor:", knn_cv.best_score_)

test_score = knn_cv.score(X_test, y_test)
print("Test seti skoru:", test_score)

In [ ]:
knn = KNeighborsClassifier(5)
knn.fit(X_train, y_train)

In [ ]:
y_pred = knn.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix (Tuned KNN)')
plt.colorbar()
sınıf = ['İyi Huylu (0)', 'Kötü Huylu (1)']
plt.xticks(np.arange(len(sınıf)), sınıf)
plt.yticks(np.arange(len(sınıf)), sınıf)
for i in range(len(sınıf)):
    for j in range(len(sınıf)):
        plt.text(j, i, str(conf_matrix[i, j]), horizontalalignment='center', color='white' if conf_matrix[i, j] > conf_matrix.max() / 2 else 'black')

plt.xlabel('Tahmin Edilen Değer')
plt.ylabel('Gerçek Değer')
plt.show()

In [ ]:
svc_model = SVC()

In [ ]:
svc_model.fit(X_train, y_train)

In [ ]:
y_pred = svc_model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix (SVC)')
plt.colorbar()
sınıf = ['İyi Huylu (0)', 'Kötü Huylu (1)']
plt.xticks(np.arange(len(sınıf)), sınıf)
plt.yticks(np.arange(len(sınıf)), sınıf)
for i in range(len(sınıf)):
    for j in range(len(sınıf)):
        plt.text(j, i, str(conf_matrix[i, j]), horizontalalignment='center', color='white' if conf_matrix[i, j] > conf_matrix.max() / 2 else 'black')

plt.xlabel('Tahmin Edilen Değer')
plt.ylabel('Gerçek Değer')
plt.show()

In [ ]:
svc_params = {'C': [0.1, 1, 10, 100], 
              'gamma': [1, 0.1, 0.01, 0.001]}

In [ ]:
svc= SVC()

In [ ]:
svc_cv = GridSearchCV(svc, svc_params, cv=5)

In [ ]:
svc_cv.fit(X_train, y_train)

In [ ]:
print("En iyi parametreler:", svc_cv.best_params_)
print("En iyi skor:", svc_cv.best_score_)

In [ ]:
test_score = svc_cv.score(X_test, y_test)
print("Test seti skoru:", test_score)

In [ ]:
svc = SVC(C = 10, gamma = 0.01)

In [ ]:
svc.fit(X_train, y_train)

In [ ]:
y_pred = svc.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix (Tuned SVC)')
plt.colorbar()
sınıf = ['İyi Huylu (0)', 'Kötü Huylu (1)']
plt.xticks(np.arange(len(sınıf)), sınıf)
plt.yticks(np.arange(len(sınıf)), sınıf)
for i in range(len(sınıf)):
    for j in range(len(sınıf)):
        plt.text(j, i, str(conf_matrix[i, j]), horizontalalignment='center', color='white' if conf_matrix[i, j] > conf_matrix.max() / 2 else 'black')

plt.xlabel('Tahmin Edilen Değer')
plt.ylabel('Gerçek Değer')
plt.show()

In [ ]:
mlpc_model = MLPClassifier()

In [ ]:
mlpc_model.fit(X_train, y_train)

In [ ]:
y_pred = mlpc_model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix (MLPC)')
plt.colorbar()
sınıf = ['İyi Huylu (0)', 'Kötü Huylu (1)']
plt.xticks(np.arange(len(sınıf)), sınıf)
plt.yticks(np.arange(len(sınıf)), sınıf)
for i in range(len(sınıf)):
    for j in range(len(sınıf)):
        plt.text(j, i, str(conf_matrix[i, j]), horizontalalignment='center', color='white' if conf_matrix[i, j] > conf_matrix.max() / 2 else 'black')

plt.xlabel('Tahmin Edilen Değer')
plt.ylabel('Gerçek Değer')
plt.show()

In [ ]:
mlpc_params = {
    'hidden_layer_sizes': [(100,), (50,), (50, 50)],
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive']
}

In [ ]:
mlpc = MLPClassifier()

In [ ]:
mlpc_cv = GridSearchCV(mlpc, mlpc_params, cv=5)

In [ ]:
mlpc_cv.fit(X_train, y_train)

In [ ]:
print("En iyi parametreler:", mlpc_cv.best_params_)
print("En iyi skor:", mlpc_cv.best_score_)
test_score = mlpc_cv.score(X_test, y_test)
print("Test seti skoru:", test_score)

In [ ]:
mlpc = MLPClassifier(activation = "tanh", alpha = 0.05, hidden_layer_sizes = (50, 50), learning_rate = "constant", solver = "adam")

In [ ]:
mlpc.fit(X_train, y_train)

In [ ]:
y_pred = mlpc.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix (Tuned MLPC)')
plt.colorbar()
sınıf = ['İyi Huylu (0)', 'Kötü Huylu (1)']
plt.xticks(np.arange(len(sınıf)), sınıf)
plt.yticks(np.arange(len(sınıf)), sınıf)
for i in range(len(sınıf)):
    for j in range(len(sınıf)):
        plt.text(j, i, str(conf_matrix[i, j]), horizontalalignment='center', color='white' if conf_matrix[i, j] > conf_matrix.max() / 2 else 'black')

plt.xlabel('Tahmin Edilen Değer')
plt.ylabel('Gerçek Değer')
plt.show()

In [ ]:
cart_model = DecisionTreeClassifier()

In [ ]:
cart_model.fit(X_train, y_train)

In [ ]:
y_pred = cart_model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix (CART)')
plt.colorbar()
sınıf = ['İyi Huylu (0)', 'Kötü Huylu (1)']
plt.xticks(np.arange(len(sınıf)), sınıf)
plt.yticks(np.arange(len(sınıf)), sınıf)
for i in range(len(sınıf)):
    for j in range(len(sınıf)):
        plt.text(j, i, str(conf_matrix[i, j]), horizontalalignment='center', color='white' if conf_matrix[i, j] > conf_matrix.max() / 2 else 'black')

plt.xlabel('Tahmin Edilen Değer')
plt.ylabel('Gerçek Değer')
plt.show()

In [ ]:
cart_params = {
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [ ]:
cart = DecisionTreeClassifier()

In [ ]:
cart_cv = GridSearchCV(cart, cart_params, cv=5)

In [ ]:
cart_cv.fit(X_train, y_train)

In [ ]:
print("En iyi parametreler:", cart_cv.best_params_)
print("En iyi skor:", cart_cv.best_score_)
test_score = cart_cv.score(X_test, y_test)
print("Test seti skoru:", test_score)

In [ ]:
cart = DecisionTreeClassifier(max_depth = 10, min_samples_leaf = 2, min_samples_split = 10)

In [ ]:
cart.fit(X_train, y_train)

In [ ]:
y_pred = cart.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix (Tuned CART)')
plt.colorbar()
sınıf = ['İyi Huylu (0)', 'Kötü Huylu (1)']
plt.xticks(np.arange(len(sınıf)), sınıf)
plt.yticks(np.arange(len(sınıf)), sınıf)
for i in range(len(sınıf)):
    for j in range(len(sınıf)):
        plt.text(j, i, str(conf_matrix[i, j]), horizontalalignment='center', color='white' if conf_matrix[i, j] > conf_matrix.max() / 2 else 'black')

plt.xlabel('Tahmin Edilen Değer')
plt.ylabel('Gerçek Değer')
plt.show()

In [ ]:
rf_model = RandomForestClassifier()

In [ ]:
rf_model.fit(X_train, y_train)

In [ ]:
y_pred = rf_model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix (Random Forest)')
plt.colorbar()
sınıf = ['İyi Huylu (0)', 'Kötü Huylu (1)']
plt.xticks(np.arange(len(sınıf)), sınıf)
plt.yticks(np.arange(len(sınıf)), sınıf)
for i in range(len(sınıf)):
    for j in range(len(sınıf)):
        plt.text(j, i, str(conf_matrix[i, j]), horizontalalignment='center', color='white' if conf_matrix[i, j] > conf_matrix.max() / 2 else 'black')

plt.xlabel('Tahmin Edilen Değer')
plt.ylabel('Gerçek Değer')
plt.show()

In [ ]:
rf_params = {
    'n_estimators': [100, 300, 500],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [ ]:
rf = RandomForestClassifier()

In [ ]:
rf_cv = GridSearchCV(rf, rf_params, cv=5)

In [ ]:
rf_cv.fit(X_train, y_train)

In [ ]:
print("En iyi parametreler:", rf_cv.best_params_)
print("En iyi skor:", rf_cv.best_score_)
test_score = rf_cv.score(X_test, y_test)
print("Test seti skoru:", test_score)

In [ ]:
rf = RandomForestClassifier(max_depth = None, min_samples_leaf = 1, min_samples_split = 2, n_estimators = 100)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix (Tuned Random Forest)')
plt.colorbar()
sınıf = ['İyi Huylu (0)', 'Kötü Huylu (1)']
plt.xticks(np.arange(len(sınıf)), sınıf)
plt.yticks(np.arange(len(sınıf)), sınıf)
for i in range(len(sınıf)):
    for j in range(len(sınıf)):
        plt.text(j, i, str(conf_matrix[i, j]), horizontalalignment='center', color='white' if conf_matrix[i, j] > conf_matrix.max() / 2 else 'black')

plt.xlabel('Tahmin Edilen Değer')
plt.ylabel('Gerçek Değer')
plt.show()

In [ ]:
gbc_model = GradientBoostingClassifier()

In [ ]:
gbc_model.fit(X_train, y_train)

In [ ]:
y_pred = gbc_model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix (GBC)')
plt.colorbar()
sınıf = ['İyi Huylu (0)', 'Kötü Huylu (1)']
plt.xticks(np.arange(len(sınıf)), sınıf)
plt.yticks(np.arange(len(sınıf)), sınıf)
for i in range(len(sınıf)):
    for j in range(len(sınıf)):
        plt.text(j, i, str(conf_matrix[i, j]), horizontalalignment='center', color='white' if conf_matrix[i, j] > conf_matrix.max() / 2 else 'black')

plt.xlabel('Tahmin Edilen Değer')
plt.ylabel('Gerçek Değer')
plt.show()

In [ ]:
gbc_params = {
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7]
}

In [ ]:
gbc = GradientBoostingClassifier()

In [ ]:
gbc_cv = GridSearchCV(gbc, gbc_params, cv=5)

In [ ]:
gbc_cv.fit(X_train, y_train)

In [ ]:
print("En iyi parametreler:", gbc_cv.best_params_)
print("En iyi skor:", gbc_cv.best_score_)
test_score = gbc_cv.score(X_test, y_test)
print("Test seti skoru:", test_score)

In [ ]:
gbc = GradientBoostingClassifier(learning_rate =  0.2, max_depth = 3, n_estimators = 100)

In [ ]:
gbc.fit(X_train, y_train)

In [ ]:
y_pred = gbc.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix (Tuned GBC)')
plt.colorbar()
sınıf = ['İyi Huylu (0)', 'Kötü Huylu (1)']
plt.xticks(np.arange(len(sınıf)), sınıf)
plt.yticks(np.arange(len(sınıf)), sınıf)
for i in range(len(sınıf)):
    for j in range(len(sınıf)):
        plt.text(j, i, str(conf_matrix[i, j]), horizontalalignment='center', color='white' if conf_matrix[i, j] > conf_matrix.max() / 2 else 'black')

plt.xlabel('Tahmin Edilen Değer')
plt.ylabel('Gerçek Değer')
plt.show()

In [ ]:
xgbc_model = XGBClassifier()

In [ ]:
xgbc_model.fit(X_train,y_train)

In [ ]:
y_pred = xgbc_model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix (XGBC)')
plt.colorbar()
sınıf = ['İyi Huylu (0)', 'Kötü Huylu (1)']
plt.xticks(np.arange(len(sınıf)), sınıf)
plt.yticks(np.arange(len(sınıf)), sınıf)
for i in range(len(sınıf)):
    for j in range(len(sınıf)):
        plt.text(j, i, str(conf_matrix[i, j]), horizontalalignment='center', color='white' if conf_matrix[i, j] > conf_matrix.max() / 2 else 'black')

plt.xlabel('Tahmin Edilen Değer')
plt.ylabel('Gerçek Değer')
plt.show()

In [ ]:
xgbc_params = {
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7]
}

In [ ]:
xgbc = XGBClassifier()

In [ ]:
xgbc_cv = GridSearchCV(xgbc, xgbc_params, cv=5)

In [ ]:
xgbc_cv.fit(X_train, y_train)

In [ ]:
print("En iyi parametreler:", xgbc_cv.best_params_)
print("En iyi skor:", xgbc_cv.best_score_)
test_score = xgbc_cv.score(X_test, y_test)
print("Test seti skoru:", test_score)

In [ ]:
xgbc = XGBClassifier(learning_rate =  0.05, max_depth = 3, n_estimators = 200)

In [ ]:
xgbc.fit(X_train, y_train)

In [ ]:
y_pred = xgbc.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix (Tuned XGBC)')
plt.colorbar()
sınıf = ['İyi Huylu (0)', 'Kötü Huylu (1)']
plt.xticks(np.arange(len(sınıf)), sınıf)
plt.yticks(np.arange(len(sınıf)), sınıf)
for i in range(len(sınıf)):
    for j in range(len(sınıf)):
        plt.text(j, i, str(conf_matrix[i, j]), horizontalalignment='center', color='white' if conf_matrix[i, j] > conf_matrix.max() / 2 else 'black')

plt.xlabel('Tahmin Edilen Değer')
plt.ylabel('Gerçek Değer')
plt.show()

In [ ]:
lgbm_model = LGBMClassifier()

In [ ]:
lgbm_model.fit(X_train,y_train)

In [ ]:
y_pred = lgbm_model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix (LightGBM)')
plt.colorbar()
sınıf = ['İyi Huylu (0)', 'Kötü Huylu (1)']
plt.xticks(np.arange(len(sınıf)), sınıf)
plt.yticks(np.arange(len(sınıf)), sınıf)
for i in range(len(sınıf)):
    for j in range(len(sınıf)):
        plt.text(j, i, str(conf_matrix[i, j]), horizontalalignment='center', color='white' if conf_matrix[i, j] > conf_matrix.max() / 2 else 'black')

plt.xlabel('Tahmin Edilen Değer')
plt.ylabel('Gerçek Değer')
plt.show()

In [ ]:
lgbm_params = {
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7]
}

In [ ]:
lgbm = LGBMClassifier()

In [ ]:
lgbm_cv = GridSearchCV(lgbm, lgbm_params, cv=5)

In [ ]:
lgbm_cv.fit(X_train, y_train)

In [ ]:
print("En iyi parametreler:", lgbm_cv.best_params_)
print("En iyi skor:", lgbm_cv.best_score_)
test_score = lgbm_cv.score(X_test, y_test)
print("Test seti skoru:", test_score)

In [ ]:
lgbm = LGBMClassifier(learning_rate =  0.2, max_depth = 3, n_estimators = 100)

In [ ]:
lgbm.fit(X_train, y_train)

In [ ]:
y_pred = lgbm.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix (Tuned LightGBM)')
plt.colorbar()
sınıf = ['İyi Huylu (0)', 'Kötü Huylu (1)']
plt.xticks(np.arange(len(sınıf)), sınıf)
plt.yticks(np.arange(len(sınıf)), sınıf)
for i in range(len(sınıf)):
    for j in range(len(sınıf)):
        plt.text(j, i, str(conf_matrix[i, j]), horizontalalignment='center', color='white' if conf_matrix[i, j] > conf_matrix.max() / 2 else 'black')

plt.xlabel('Tahmin Edilen Değer')
plt.ylabel('Gerçek Değer')
plt.show()

In [ ]:
catb_model = CatBoostClassifier()

In [ ]:
catb_model.fit(X_train,y_train)

In [ ]:
y_pred = catb_model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix (CatBoost)')
plt.colorbar()
sınıf = ['İyi Huylu (0)', 'Kötü Huylu (1)']
plt.xticks(np.arange(len(sınıf)), sınıf)
plt.yticks(np.arange(len(sınıf)), sınıf)
for i in range(len(sınıf)):
    for j in range(len(sınıf)):
        plt.text(j, i, str(conf_matrix[i, j]), horizontalalignment='center', color='white' if conf_matrix[i, j] > conf_matrix.max() / 2 else 'black')

plt.xlabel('Tahmin Edilen Değer')
plt.ylabel('Gerçek Değer')
plt.show()

In [ ]:
catb_params = {
    'learning_rate': [0.05, 0.1, 0.2],
    'iterations': [50, 100, 200],
    'depth': [3, 5, 7]
}

In [ ]:
catb = CatBoostClassifier()

In [ ]:
catb_cv = GridSearchCV(catb, catb_params, cv=5)

In [ ]:
catb_cv.fit(X_train, y_train)

In [ ]:
print("En iyi parametreler:", catb_cv.best_params_)
print("En iyi skor:", catb_cv.best_score_)
test_score = catb_cv.score(X_test, y_test)
print("Test seti skoru:", test_score)

In [ ]:
catb = CatBoostClassifier(depth = 3, iterations = 100, learning_rate = 0.05)

In [ ]:
catb.fit(X_train, y_train)

In [ ]:
y_pred = catb.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix (Tuned CatBoost)')
plt.colorbar()
sınıf = ['İyi Huylu (0)', 'Kötü Huylu (1)']
plt.xticks(np.arange(len(sınıf)), sınıf)
plt.yticks(np.arange(len(sınıf)), sınıf)
for i in range(len(sınıf)):
    for j in range(len(sınıf)):
        plt.text(j, i, str(conf_matrix[i, j]), horizontalalignment='center', color='white' if conf_matrix[i, j] > conf_matrix.max() / 2 else 'black')

plt.xlabel('Tahmin Edilen Değer')
plt.ylabel('Gerçek Değer')
plt.show()

In [ ]:
modeller = [
    loj_model,
    nb_model,
    knn,
    svc,
    mlpc,
    cart,
    rf,
    gbc,
    xgbc,
    lgbm,
    catb,
]
f1_scores = {}

for model in modeller:
    isimler = model.__class__.__name__
    y_pred = model.predict(X_test)
    f1_score = classification_report(y_test, y_pred, output_dict=True)['weighted avg']['f1-score']
    f1_scores[isimler] = f1_score
for isimler, f1_score in f1_scores.items():
    print(f"{isimler}: F1 Score: {f1_score:.4f}")



In [ ]:
sonuclar_list = []

for model in modeller:
    isimler = model.__class__.__name__
    y_pred = model.predict(X_test)
    dogruluk = accuracy_score(y_test, y_pred)
    sonuc = pd.DataFrame([[isimler, dogruluk * 100]], columns=["Modeller", "Accuracy"])
    sonuclar_list.append(sonuc)
sonuclar = pd.concat(sonuclar_list, ignore_index=True)

In [ ]:
sonuclar = sonuclar.sort_values(by='Accuracy', ascending=True)
unique_accuracy_values = sonuclar['Accuracy'].unique()
colors = sns.color_palette('viridis', len(unique_accuracy_values))
color_dict = {acc: color for acc, color in zip(unique_accuracy_values, colors)}
plt.figure(figsize=(10, 6))
for i, row in sonuclar.iterrows():
    plt.barh(row['Modeller'], row['Accuracy'], color=color_dict[row['Accuracy']])
plt.xlabel('Accuracy%')
plt.title('Modellerin Doğruluk Oranları')
plt.show()

In [ ]:
# X = df[['concavity_mean']]  
# y = df['target']  
# model = gbm_tuned 
# model.fit(X, y)  
# value = 0.546
# predicted_value = model.predict([[value]])  
# print(predicted_value)

In [ ]:
from sklearn.model_selection import learning_curve
train_sizes, train_scores, test_scores = learning_curve(loj_model, X, y, train_sizes=np.linspace(0.1, 1.0, 10), cv=5)

# Learning curve'ü çizme
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)

plt.figure()
plt.title("Learning Curve")
plt.xlabel("Training examples")
plt.ylabel("Score")

plt.grid()

plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")

plt.legend(loc="best")

plt.show()

In [ ]:
importance = pd.DataFrame({"Importance": np.abs(loj_model.coef_[0])}, index=X.columns)

# Değişken önem düzeylerini sıralama
importance_sorted = importance.sort_values(by="Importance", ascending=True)
colors = plt.cm.viridis(np.linspace(0, 1, len(importance_sorted)))

# Değişken önem düzeylerini görselleştirme
plt.figure(figsize=(10, 6))
plt.barh(importance_sorted.index, importance_sorted["Importance"], color=colors)
plt.xlabel("Değişken Önem Düzeyleri")
plt.title("Lojistik Regresyon Değişken Önemleri")
plt.show()